In [1]:
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_NOTEBOOK
# CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

In [2]:

from pyecharts.charts import Bar
from pyecharts import options as opts
from pyecharts.globals import ThemeType

bar =(
    Bar(init_opts=opts.InitOpts(theme=ThemeType.WESTEROS,width="100%", height="400px"))
    .add_xaxis(["衬衫","羊毛衫","雪纺衫","裤子","高跟鞋","袜子"])
    .add_yaxis("商家A",[5, 20, 36,10, 75, 90])
    .add_yaxis("商家B",[15,6,45, 20,35, 66])
    .set_global_opts(
        title_opts=opts.TitleOpts(title="主标题",subtitle="副标题"),
                        datazoom_opts=[
                            opts.DataZoomOpts(type_="slider"),     # 底部滑块
                            opts.DataZoomOpts(type_="inside")      # 鼠标滚轮缩放（内嵌）
                        ],
                    )
    )

# bar.load_javascript()
bar.render_notebook()

In [20]:
# import pyecharts.options as opts
from pyecharts.charts import Line


x_data = ["周一", "周二", "周三", "周四", "周五", "周六", "周日"]
y_data = [820, 932, 901, 934, 1290, 1330, 1320]


line=(
    Line(init_opts=opts.InitOpts(theme=ThemeType.WESTEROS,width="100%", height="600px"))
    .add_xaxis(xaxis_data=x_data)
    .add_yaxis(
        series_name="邮件营销",
        stack="总量",
        y_axis=[120, 132, 101, 134, 90, 230, 210],
        areastyle_opts=opts.AreaStyleOpts(opacity=0.5),
        label_opts=opts.LabelOpts(is_show=False),
    )
    .add_yaxis(
        series_name="联盟广告",
        stack="总量",
        y_axis=[220, 182, 191, 234, 290, 330, 310],
        areastyle_opts=opts.AreaStyleOpts(opacity=0.5),
        label_opts=opts.LabelOpts(is_show=False),
    )
    .add_yaxis(
        series_name="视频广告",
        stack="总量",
        y_axis=[150, 232, 201, 154, 190, 330, 410],
        areastyle_opts=opts.AreaStyleOpts(opacity=0.5),
        label_opts=opts.LabelOpts(is_show=False),
    )
    .add_yaxis(
        series_name="直接访问",
        stack="总量",
        y_axis=[320, 332, 301, 334, 390, 330, 320],
        areastyle_opts=opts.AreaStyleOpts(opacity=0.5),
        label_opts=opts.LabelOpts(is_show=False),
    )
    .add_yaxis(
        series_name="搜索引擎",
        stack="总量",
        y_axis=[820, 932, 901, 934, 1290, 1330, 1320],
        areastyle_opts=opts.AreaStyleOpts(opacity=0.5),
        label_opts=opts.LabelOpts(is_show=True, position="top"),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="堆叠区域图"),
        tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross"),
        yaxis_opts=opts.AxisOpts(
            type_="value",
            axistick_opts=opts.AxisTickOpts(is_show=True),
            splitline_opts=opts.SplitLineOpts(is_show=True),
        ),
        xaxis_opts=opts.AxisOpts(type_="category", boundary_gap=False),
        datazoom_opts=[
            opts.DataZoomOpts(type_="slider"),     # 底部滑块
            opts.DataZoomOpts(type_="inside")      # 鼠标滚轮缩放（内嵌）
        ],
    )
    
)
line.render_notebook()


In [21]:
import pandas as pd
from pyecharts.charts import Line
from pyecharts import options as opts
from IPython.display import display, HTML
import ipywidgets as widgets
import io

# 全局变量
data_raw = pd.DataFrame()
data_df = pd.DataFrame()
header_row_index = 0
data_end_row_index = 0

# 上传按钮
upload_button = widgets.FileUpload(
    accept='.csv,.xlsx',
    multiple=False,
    description='上传表格',
    button_style='info'
)

# 标题行 / 结束行选择
start_row_dropdown = widgets.Dropdown(description="标题行：")
end_row_dropdown = widgets.Dropdown(description="结束行：")

# x轴 和 y轴选择框
x_dropdown = widgets.Dropdown(description="X 轴：")
y_dropdowns = [widgets.Dropdown(description=f"Y 轴 {i+1}：") for i in range(5)]

# 输出区域
output_chart = widgets.Output()

# 处理上传文件
def on_upload_change(change):
    global data_raw
    output_chart.clear_output()

    if upload_button.value:
        # uploaded_file = list(upload_button.value.values())[0]
        # content = uploaded_file['content']
        # filename = uploaded_file['metadata']['name']

        # 获取上传文件内容（兼容 ipywidgets 7.x 和 8.x）
        if isinstance(upload_button.value, dict):  # ipywidgets 7.x
            uploaded_file = next(iter(upload_button.value.values()))
            filename = list(upload_button.value.keys())[0]
            content = uploaded_file['content']
        elif isinstance(upload_button.value, tuple):  # ipywidgets 8.x
            uploaded_file = upload_button.value[0]
            filename = uploaded_file['name']
            content = uploaded_file['content']
        else:
            with output_chart:
                print("无法识别上传文件格式")
                return

        try:
            if filename.endswith('.csv'):
                data_raw = pd.read_csv(io.BytesIO(content), header=None)
            elif filename.endswith('.xlsx'):
                data_raw = pd.read_excel(io.BytesIO(content), header=None, engine='openpyxl')
        except Exception as e:
            with output_chart:
                print("读取失败:", e)
            return

        row_count = len(data_raw)
        options = [(f"第 {i+1} 行", i) for i in range(row_count)]
        start_row_dropdown.options = options
        end_row_dropdown.options = options

        with output_chart:
            print("文件读取成功，共", row_count, "行")

upload_button.observe(on_upload_change, names='value')

# 更新列名下拉选框
def update_column_selectors():
    global data_df

    try:
        header_index = start_row_dropdown.value
        end_index = end_row_dropdown.value

        if header_index is None or end_index is None:
            return

        if header_index >= end_index:
            with output_chart:
                print("标题行必须在结束行之前")
            return

        # 提取标题和数据
        headers = data_raw.iloc[header_index].tolist()
        data_df = data_raw.iloc[header_index + 1: end_index + 1].copy()
        data_df.columns = headers
        data_df.reset_index(drop=True, inplace=True)

        # 更新下拉框选项
        x_dropdown.options = headers
        for yd in y_dropdowns:
            yd.options = [""] + headers

        update_chart()

    except Exception as e:
        with output_chart:
            print("更新失败:", e)

start_row_dropdown.observe(lambda x: update_column_selectors(), names='value')
end_row_dropdown.observe(lambda x: update_column_selectors(), names='value')

# 绘制图表
def update_chart(*args):
    output_chart.clear_output()
    if data_df.empty:
        return

    x_col = x_dropdown.value
    y_cols = [yd.value for yd in y_dropdowns if yd.value]

    if not x_col or not y_cols:
        return

    try:
        x_data = data_df[x_col].astype(str).tolist()
        line = Line().add_xaxis(x_data)

        for y in y_cols:
            y_data = data_df[y].astype(float).tolist()
            line.add_yaxis(y, y_data)

        line.set_global_opts(
            title_opts=opts.TitleOpts(title="曲线图"),
            tooltip_opts=opts.TooltipOpts(trigger="axis"),
            toolbox_opts=opts.ToolboxOpts()
        )



        # with output_chart:
        #     # display(HTML(line.render_embed()))
        #     line.render_notebook()
        with output_chart:
            display(line.render_notebook())

    except Exception as e:
        with output_chart:
            print("绘图失败：", e)

# 给下拉绑定图表刷新事件
x_dropdown.observe(update_chart, names='value')
for yd in y_dropdowns:
    yd.observe(update_chart, names='value')

# 显示控件
display(upload_button)
display(widgets.HBox([start_row_dropdown, end_row_dropdown]))
display(widgets.HBox([x_dropdown]))
display(widgets.HBox(y_dropdowns))
display(output_chart)


FileUpload(value=(), accept='.csv,.xlsx', button_style='info', description='上传表格')

Output()

In [22]:
import pandas as pd
from pyecharts.charts import Line
from pyecharts import options as opts
from IPython.display import display, HTML
import ipywidgets as widgets
import io

# 全局变量
data_raw = pd.DataFrame()
data_df = pd.DataFrame()
header_row_index = 0
data_end_row_index = 0

# 上传按钮
upload_button = widgets.FileUpload(
    accept='.csv,.xlsx',
    multiple=False,
    description='上传表格',
    button_style='info'
)

# 标题行 / 结束行选择
start_row_dropdown = widgets.Dropdown(description="标题行：")
end_row_dropdown = widgets.Dropdown(description="结束行：")

# x轴 和 y轴选择框
x_dropdown = widgets.Dropdown(description="X 轴：")
y_dropdowns = [widgets.Dropdown(description=f"Y 轴 {i+1}：") for i in range(5)]

# 输出区域
output_chart = widgets.Output()

# 处理上传文件
def on_upload_change(change):
    global data_raw
    output_chart.clear_output()

    if upload_button.value:
        # uploaded_file = list(upload_button.value.values())[0]
        # content = uploaded_file['content']
        # filename = uploaded_file['metadata']['name']

        # 获取上传文件内容（兼容 ipywidgets 7.x 和 8.x）
        if isinstance(upload_button.value, dict):  # ipywidgets 7.x
            uploaded_file = next(iter(upload_button.value.values()))
            filename = list(upload_button.value.keys())[0]
            content = uploaded_file['content']
        elif isinstance(upload_button.value, tuple):  # ipywidgets 8.x
            uploaded_file = upload_button.value[0]
            filename = uploaded_file['name']
            content = uploaded_file['content']
        else:
            with output_chart:
                print("无法识别上传文件格式")
                return

        try:
            if filename.endswith('.csv'):
                data_raw = pd.read_csv(io.BytesIO(content), header=None)
            elif filename.endswith('.xlsx'):
                data_raw = pd.read_excel(io.BytesIO(content), header=None, engine='openpyxl')
        except Exception as e:
            with output_chart:
                print("读取失败:", e)
            return

        row_count = len(data_raw)
        options = [(f"第 {i+1} 行", i) for i in range(row_count)]
        start_row_dropdown.options = options
        end_row_dropdown.options = options

        with output_chart:
            print("文件读取成功，共", row_count, "行")

upload_button.observe(on_upload_change, names='value')

# 更新列名下拉选框
def update_column_selectors():
    global data_df

    try:
        header_index = start_row_dropdown.value
        end_index = end_row_dropdown.value

        if header_index is None or end_index is None:
            return

        if header_index >= end_index:
            with output_chart:
                print("标题行必须在结束行之前")
            return

        # 提取标题和数据
        headers = data_raw.iloc[header_index].tolist()
        data_df = data_raw.iloc[header_index + 1: end_index + 1].copy()
        data_df.columns = headers
        data_df.reset_index(drop=True, inplace=True)

        # 更新下拉框选项
        x_dropdown.options = headers
        for yd in y_dropdowns:
            yd.options = [""] + headers

        update_chart()

    except Exception as e:
        with output_chart:
            print("更新失败:", e)

start_row_dropdown.observe(lambda x: update_column_selectors(), names='value')
end_row_dropdown.observe(lambda x: update_column_selectors(), names='value')

# 新增：全屏切换按钮
fullscreen_toggle = widgets.ToggleButton(
    value=False,
    description='全屏显示',
    button_style='success',
    tooltip='切换图表全屏显示 / 恢复',
    icon='arrows-alt'
)

def update_chart(*args):
    output_chart.clear_output()
    if data_df.empty:
        return

    x_col = x_dropdown.value
    y_cols = [yd.value for yd in y_dropdowns if yd.value]

    if not x_col or not y_cols:
        return

    try:
        x_data = data_df[x_col].astype(str).tolist()
        

        # 根据全屏开关调整尺寸
        if fullscreen_toggle.value:
            # 全屏模式，使用视口单位设置较大尺寸
            width = "90vw"
            height = "80vh"
        else:
            # 默认模式，宽度自适应，但限制最大宽高
            width = "100%"
            height = "400px"

        # pyecharts 直接传入宽高
        line = Line(init_opts=opts.InitOpts(width=width, height=height))
        # line = line.init_opts(opts.InitOpts(width=width, height=height))
        line.add_xaxis(x_data)

        for y in y_cols:
            y_data = data_df[y].astype(float).tolist()
            line.add_yaxis(y, y_data)

        line.set_global_opts(
            title_opts=opts.TitleOpts(title="曲线图"),
            tooltip_opts=opts.TooltipOpts(trigger="axis"),
            toolbox_opts=opts.ToolboxOpts()
        )

        with output_chart:
            display(line.render_notebook())

    except Exception as e:
        with output_chart:
            print("绘图失败：", e)

# 监听全屏按钮
fullscreen_toggle.observe(update_chart, names='value')

# 其他控件绑定图表更新事件
x_dropdown.observe(update_chart, names='value')
for yd in y_dropdowns:
    yd.observe(update_chart, names='value')

# 初始绘制
update_chart()

# 显示控件（增加全屏切换按钮）
display(upload_button)
display(widgets.HBox([start_row_dropdown, end_row_dropdown]))
display(widgets.HBox([x_dropdown]))
display(widgets.HBox(y_dropdowns))
display(fullscreen_toggle)
display(output_chart)

FileUpload(value=(), accept='.csv,.xlsx', button_style='info', description='上传表格')

ToggleButton(value=False, button_style='success', description='全屏显示', icon='arrows-alt', tooltip='切换图表全屏显示 / 恢…

Output()

In [23]:
import pandas as pd
from pyecharts.charts import Line
from pyecharts import options as opts
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
import io

# 全局变量
data_raw = pd.DataFrame()
data_df = pd.DataFrame()
selected_index = None

# 上传控件
upload_button = widgets.FileUpload(
    accept='.csv,.xlsx',
    multiple=False,
    description='上传表格',
    button_style='info'
)

start_row_dropdown = widgets.Dropdown(description="标题行：")
end_row_dropdown = widgets.Dropdown(description="结束行：")
x_dropdown = widgets.Dropdown(description="X 轴：")
y_dropdowns = [widgets.Dropdown(description=f"Y 轴 {i+1}：") for i in range(5)]

output_chart = widgets.Output()
output_table = widgets.Output()
row_slider = widgets.IntSlider(description='选中行', min=0, value=0)

# 处理上传文件
def on_upload_change(change):
    global data_raw
    output_chart.clear_output()
    output_table.clear_output()

    if upload_button.value:
        if isinstance(upload_button.value, dict):  # ipywidgets 7.x
            uploaded_file = next(iter(upload_button.value.values()))
            filename = list(upload_button.value.keys())[0]
            content = uploaded_file['content']
        elif isinstance(upload_button.value, tuple):  # ipywidgets 8.x
            uploaded_file = upload_button.value[0]
            filename = uploaded_file['name']
            content = uploaded_file['content']
        else:
            with output_chart:
                print("无法识别上传文件格式")
            return

        try:
            if filename.endswith('.csv'):
                data_raw = pd.read_csv(io.BytesIO(content), header=None)
            elif filename.endswith('.xlsx'):
                data_raw = pd.read_excel(io.BytesIO(content), header=None, engine='openpyxl')
        except Exception as e:
            with output_chart:
                print("读取失败:", e)
            return

        row_count = len(data_raw)
        options = [(f"第 {i+1} 行", i) for i in range(row_count)]
        start_row_dropdown.options = options
        end_row_dropdown.options = options
        row_slider.max = row_count - 1

        with output_chart:
            print("文件读取成功，共", row_count, "行")

upload_button.observe(on_upload_change, names='value')

# 提取数据 & 更新下拉
def update_column_selectors():
    global data_df
    try:
        header_index = start_row_dropdown.value
        end_index = end_row_dropdown.value

        if header_index is None or end_index is None or header_index >= end_index:
            return

        headers = data_raw.iloc[header_index].tolist()
        data_df = data_raw.iloc[header_index + 1: end_index + 1].copy()
        data_df.columns = headers
        data_df.reset_index(drop=True, inplace=True)

        x_dropdown.options = headers
        for yd in y_dropdowns:
            yd.options = [""] + headers

        row_slider.max = len(data_df) - 1
        update_table()
        update_chart()

    except Exception as e:
        with output_chart:
            print("更新失败:", e)

start_row_dropdown.observe(lambda x: update_column_selectors(), names='value')
end_row_dropdown.observe(lambda x: update_column_selectors(), names='value')

# 表格预览更新
def update_table():
    output_table.clear_output()
    if data_df.empty:
        return
    with output_table:
        display(data_df.style.apply(highlight_row, axis=1))

# 高亮函数
def highlight_row(row):
    return ['background-color: lightyellow' if row.name == selected_index else '' for _ in row]

# 图表更新
def update_chart(*args):
    output_chart.clear_output()
    if data_df.empty:
        return

    x_col = x_dropdown.value
    y_cols = [yd.value for yd in y_dropdowns if yd.value]

    if not x_col or not y_cols:
        return

    try:
        x_data = data_df[x_col].astype(str).tolist()
        line = Line().add_xaxis(x_data)

        for y in y_cols:
            y_data = data_df[y].astype(float).tolist()
            line.add_yaxis(
                y, y_data,
                is_selected=True,
                markpoint_opts=opts.MarkPointOpts(
                    data=[opts.MarkPointItem(type_="max", name="最大值")]
                ),
                label_opts=opts.LabelOpts(is_show=False),
                itemstyle_opts=opts.ItemStyleOpts(
                    color="red" if selected_index is not None else None
                )
            )

        # 高亮样式：用 axisPointer + tooltip 显示选中位置
        line.set_global_opts(
            title_opts=opts.TitleOpts(title="曲线图"),
            tooltip_opts=opts.TooltipOpts(trigger="axis"),
            toolbox_opts=opts.ToolboxOpts(),
            datazoom_opts=[opts.DataZoomOpts()],
            xaxis_opts=opts.AxisOpts(
                axispointer_opts=opts.AxisPointerOpts(
                    type_="line",
                    value=x_data[selected_index] if selected_index is not None else None,
                    label=opts.LabelOpts(formatter="{value}")
                )
            )
        )

        with output_chart:
            display(line.render_notebook())

    except Exception as e:
        with output_chart:
            print("绘图失败：", e)

# 表格与图表联动
def on_slider_change(change):
    global selected_index
    selected_index = change['new']
    update_table()
    update_chart()

x_dropdown.observe(update_chart, names='value')
for yd in y_dropdowns:
    yd.observe(update_chart, names='value')
row_slider.observe(on_slider_change, names='value')

# 显示 UI
ui_layout = widgets.HBox([
    widgets.VBox([
        upload_button,
        widgets.HBox([start_row_dropdown, end_row_dropdown]),
        widgets.HBox([x_dropdown]),
        widgets.HBox(y_dropdowns),
        row_slider,
        output_table
    ], layout=widgets.Layout(width='45%')),
    output_chart
])

display(ui_layout)
